# T5 finetuning with new size dataset

2025-04-23 16:18

In [1]:
import shutil
!pip install -U git+https://github.com/huggingface/transformers.git@13493215abceafc1653af88b045120014fb4c1fc
!pip install -U git+https://github.com/huggingface/accelerate.git@8ade23cc6aec7c3bd3d80fef6378cafaade75bbe
!pip install -U git+https://github.com/huggingface/datasets.git@01f91bae037c98f2e05456287bab21470adb8f07
!pip install -U git+https://github.com/huggingface/evaluate.git@55f1bc6e072b05c2d9db1589a07e20f38902b1ec
!pip install -U git+https://github.com/huggingface/safetensors.git@f5839b6aee407652aa3078d91206b618dd84e3c2

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

  Cloning https://github.com/huggingface/transformers.git (to revision 13493215abceafc1653af88b045120014fb4c1fc) to /tmp/pip-req-build-8_qr8ghf
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8_qr8ghf
  Running command git rev-parse -q --verify 'sha^13493215abceafc1653af88b045120014fb4c1fc'
  Running command git fetch -q https://github.com/huggingface/transformers.git 13493215abceafc1653af88b045120014fb4c1fc
  Running command git checkout -q 13493215abceafc1653af88b045120014fb4c1fc
  Resolved https://github.com/huggingface/transformers.git to commit 13493215abceafc1653af88b045120014fb4c1fc
  Installing build dependencies ... one
  Getting requirements to build wheel ... one
  Preparing metadata (pyproject.toml) ... one
  Cloning https://github.com/huggingface/accelerate.git (to revision 8ade23cc6aec7c3bd3d80fef6378cafaade75bbe) to /tmp/pip-req-build-ffmg9ony
  Running command git clone --filter=blob:none --quiet 

In [2]:
!apt install zip -y
!rm -rf data-rb-size
!mkdir -p data-rb-size
!wget "https://www.dropbox.com/scl/fi/t3yni1506szbqgzd50zsk/data-rb-size2.zip?rlkey=ko2kdesucccnk732hga4i94yv&dl=1" -O model.zip
!unzip model.zip -d data-rb-size

!mkdir -p data-rb-validate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 51 not upgraded.
--2025-04-23 11:41:31--  https://www.dropbox.com/scl/fi/t3yni1506szbqgzd50zsk/data-rb-size2.zip?rlkey=ko2kdesucccnk732hga4i94yv&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
connected. to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... 
HTTP request sent, awaiting response... 302 Found
Location: https://uc0c6bb69d23327975481a9e77ef.dl.dropboxusercontent.com/cd/0/inline/CoUwUAY0ETYbpQFkbHY46qgyltHRjRpBELlda99yqtiflJH2fVpjK6S0Z-HYmh40PnHZvsKAxNzHje2yRKIFgQ7bcYFLpZWuHvJwlJEdNf-0CXnICNmUoAb0jZf3cjqfiOI/file?dl=1# [following]
--2025-04-23 11:41:32--  https://uc0c6bb69d23327975481a9e77ef.dl.dropboxusercontent.com/cd/0/inline/CoUwUAY0ETYbpQFkbHY46qgyltHRjRpBELlda99yqtiflJH2fVpjK6S0Z-HYmh40PnHZvsKAxNzHje2yRKIFgQ7bcYFLpZWuHvJwlJEdNf

In [3]:
from datasets import load_from_disk, DatasetDict

validation_size = 4
test_size = 100

dataset = load_from_disk('data-rb-size')

train_test_split = dataset.train_test_split(test_size=test_size/len(dataset))
train_valid_split = train_test_split['train'].train_test_split(test_size=validation_size/len(dataset))

dataset = DatasetDict({
    "train": train_valid_split["train"],
    "valid": train_valid_split["test"],
    "test": train_test_split["test"],
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 99685
    })
    valid: Dataset({
        features: ['svg', 'html'],
        num_rows: 4
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 100
    })
})


In [4]:
print(dataset['train'][0])

{'svg': '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="div1 style1"><g data-tag="div" id="div1" data-z-index="auto" data-stacking-context="true"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="245.65625" height="852" x="0" y="0" fill="rgb(0, 0, 0)"/></g></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>', 'html': '<body><div></div></body>\n\n<style>\n\n\n        body {\n            margin: 0;\n        }\n\n        div {\n            background-color: black;\n            position: absolute;\n            height: 100vh;\n            width: 6

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
new_words = ['<', '>', '{', '}', '<body>', '<div>', '</div>', '<style>', '</style>', '</body>']
tokenizer.add_tokens(new_words)

10

In [6]:
tokenized_output = tokenizer.encode(dataset['train'][0]['svg'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized SVG contains unknown tokens.")
else:
    print("No unknown tokens found in the SVG.")

tokenized_output = tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized HTML contains unknown tokens.")
else:
    print("No unknown tokens found in the HTML.")

No unknown tokens found in the SVG.
No unknown tokens found in the HTML.


In [7]:
print(tokenizer.decode(tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)))

<body><div></div></body><style> body{ margin: 0;} div{ background-color: black; position: absolute; height: 100vh; width: 62.51%; top: 0; left: 0;}</style>


In [8]:
MAX_SEQUENCE_LENGTH = 512

def get_token_lengths(examples):
    inputs = tokenizer(
        examples['svg'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )
    labels = tokenizer(
        examples['html'],
        truncation=False,
        padding=False,
        return_length=True,
    )

    inputs["input_length"] = inputs["length"]
    inputs["labels"] = labels["input_ids"]
    inputs["label_length"] = labels["length"]

    inputs.pop("length", None)

    return inputs

tokenized_data = dataset.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['input_length'] <= MAX_SEQUENCE_LENGTH and example['label_length'] <= MAX_SEQUENCE_LENGTH

filtered_data = tokenized_data.filter(filter_function)

def tokenize_examples(examples):
    inputs = tokenizer(
        examples['svg'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    labels = tokenizer(
        examples['html'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    labels = labels['input_ids']
    labels[labels == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels
    

    return inputs

filtered_data = filtered_data.map(get_token_lengths, batched=True, remove_columns=dataset["train"].column_names)

print(filtered_data)

Map:   0%|          | 0/99685 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/99685 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/99685 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 99685
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 4
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 100
    })
})


In [9]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch
from types import SimpleNamespace

VIEWPORT_SIZES = {
    'DESKTOP': {'width': 1440, 'height': 900},
    'TABLET': {'width': 834, 'height': 1210},
    'MOBILE': {'width': 393, 'height': 852},
}

VIEWPORTS = ['MOBILE']

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def create_compute_metrics(trainer, save_best_model_dir = None):
    best_perceptual_loss = None

    def compute_metrics(eval_predictions, to_tensorboard = True):
        nonlocal best_perceptual_loss

        predictions, labels = eval_predictions
        
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)
    
        similarity_scores = []
        perceptual_losses = []
        index = 1
        for prediction, label in zip(decoded_predictions, decoded_labels):
            if to_tensorboard:
                add_text_to_tensorboard(f'valid_{index}_label_text', trainer.state.global_step, label[0])
                add_text_to_tensorboard(f'valid_{index}_prediction_text', trainer.state.global_step, prediction)

            for viewport in VIEWPORTS:
            
                metrics = calculate_metrics(prediction, label[0],
                                        VIEWPORT_SIZES[viewport]['width'], VIEWPORT_SIZES[viewport]['height'])

                if metrics is not None:
                    similarity_scores.append(metrics['similarity'])
                    perceptual_losses.append(metrics['perceptual_loss'])

                    if to_tensorboard:
                        add_image_to_tensorboard(f'valid_{index}_expectation_' + viewport.lower(), trainer.state.global_step, metrics['expected_screenshot_path'])
                        add_image_to_tensorboard(f'valid_{index}_prediction_' + viewport.lower(), trainer.state.global_step, metrics['predicted_screenshot_path'])
            
            index += 1


        result = {
            'similarity': float(np.mean(similarity_scores)),
            'perceptual_loss': float(np.mean(perceptual_losses)),
        }

        if save_best_model_dir is not None:
            if best_perceptual_loss is None or result['perceptual_loss'] < best_perceptual_loss:
                if os.path.exists(save_best_model_dir):
                    shutil.rmtree(save_best_model_dir)
                trainer.save_model(save_best_model_dir)
                tokenizer.save_pretrained(save_best_model_dir)
                with open(os.path.join(save_best_model_dir, 'best_results.txt'), 'w') as f:
                    f.write(str(result) + '\nStep: ' + str(trainer.state.global_step))

                best_perceptual_loss = result['perceptual_loss']
    
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    
    return compute_metrics

def eval_outside_of_training(model, dataset, to_tensorboard = False, step = None):
    def pad_to_matrix(seqs, pad_value=0):
        max_len = max(len(s) for s in seqs)
        matrix  = np.full((len(seqs), max_len), pad_value, dtype=np.int64)
        for i, s in enumerate(seqs):
            matrix[i, :len(s)] = s                 # left-align, pad on the right
        return matrix
    
    model.eval()
    model.to('cuda:0')

    predictions = []
    labels = []
    for row in dataset:
        inputs = tokenizer(row['svg'], return_tensors="pt").input_ids
        labels.append(tokenizer(row['html'], return_tensors="pt").input_ids)
        inputs = inputs.to('cuda:0')
        outputs = model.generate(inputs)
        predictions.append(outputs[0])

    mock_trainer = SimpleNamespace(state=SimpleNamespace(global_step=step))
    compute_metrics = create_compute_metrics(mock_trainer)
    return compute_metrics((
            pad_to_matrix([t.cpu().numpy() for t in predictions]), 
            pad_to_matrix([t.squeeze(0).cpu().numpy() for t in labels])
        ), to_tensorboard=to_tensorboard)



In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google-t5/t5-small")

In [11]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")
model.resize_token_embeddings(len(tokenizer))

Embedding(32109, 512)

In [12]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_model_config(model.config)
generation_config._from_model_config = False
generation_config.max_new_tokens = 512

generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 512,
  "pad_token_id": 0
}

In [13]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_output_dir = f"model-{timestamp}"
log_dir = f'{model_output_dir}/runs'

In [14]:
results = eval_outside_of_training(model, dataset["valid"], True, 0)
writer = SummaryWriter(log_dir=log_dir)   # or reuse the one you have
for k, v in results.items():
    writer.add_scalar(f"eval/{k}", v, 0)

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import os

training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=16,
    # gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_steps=25,
    logging_dir=log_dir,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    fp16=True,
    generation_config=generation_config,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data["train"],
    eval_dataset=filtered_data["valid"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.compute_metrics = create_compute_metrics(trainer, os.path.join(model_output_dir, "best"))

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Similarity,Perceptual Loss,Runtime,Samples Per Second,Steps Per Second
1000,0.980100,0.919178,0.633000,0.419000,22.934500,0.174000,0.044000
2000,0.785400,0.693060,0.710800,0.441400,23.471500,0.170000,0.043000
3000,0.630100,0.535017,0.753100,0.396900,23.480500,0.170000,0.043000
4000,0.525300,0.435579,0.862100,0.253400,22.380600,0.179000,0.045000
5000,0.443500,0.364305,0.976600,0.097400,22.302100,0.179000,0.045000
6000,0.376100,0.306861,0.989400,0.066900,22.321800,0.179000,0.045000
7000,0.327000,0.266441,0.996700,0.042400,21.719000,0.184000,0.046000
8000,0.294000,0.236422,0.996700,0.042400,22.299700,0.179000,0.045000
9000,0.265300,0.214546,0.997600,0.039200,21.707500,0.184000,0.046000
10000,0.244600,0.200719,0.998200,0.038500,21.926000,0.182000,0.046000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=62310, training_loss=0.23242944150115644, metrics={'train_runtime': 4559.9148, 'train_samples_per_second': 218.612, 'train_steps_per_second': 13.665, 'total_flos': 1.0362703368654029e+17, 'train_loss': 0.23242944150115644, 'epoch': 10.0})

In [16]:
import os

trainer.save_model(os.path.join(model_output_dir, "final"))
tokenizer.save_pretrained(os.path.join(model_output_dir, "final"))

('model-2025-04-23_11-42-37/final/tokenizer_config.json',
 'model-2025-04-23_11-42-37/final/special_tokens_map.json',
 'model-2025-04-23_11-42-37/final/tokenizer.json')

In [17]:
from transformers import T5ForConditionalGeneration

text = dataset['test'][0]['svg']

inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to('cuda:0')

model = T5ForConditionalGeneration.from_pretrained(os.path.join(model_output_dir, "best"))
model.to('cuda:0')
model.generation_config = generation_config

outputs = model.generate(inputs)
# outputs = model.generate(inputs, max_new_tokens=16_384, do_sample=True, top_k=30, top_p=0.95)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(dataset['test'][0]['html'])
print(translation)


<body><div></div></body>

<style>


        body {
            margin: 0;
        }

        div {
            background-color: black;
            position: absolute;
            height: 100vh;
            width: 31.166%;
            top: 0;
            left: 0;
        }


</style>
<body><div></div></body><style> body{ margin: 0;} div{ background-color: black; position: absolute; height: 100vh; width: 31.067%; top: 0; left: 0;}</style>


In [18]:
test_results = eval_outside_of_training(model, dataset["test"], False)

print(test_results)

{'similarity': 0.9985, 'perceptual_loss': 0.0318}
